In [16]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder

In [17]:
# 이미 커널에 객체가 있다면 이 셀은 그냥 통과합니다.
required = ["final_model", "X_train", "y_train", "X_test", "y_test"]
missing = [name for name in required if name not in globals()]
if missing:
    print("다음 객체가 현재 커널에 없습니다:", missing)
    print("→ 먼저 학습을 실행하여 위 객체를 메모리에 올리거나, 저장해둔 파일에서 로드하세요.")
else:
    print("모든 학습 산출물이 메모리에 있습니다:", required)

다음 객체가 현재 커널에 없습니다: ['final_model', 'X_train', 'y_train', 'X_test', 'y_test']
→ 먼저 학습을 실행하여 위 객체를 메모리에 올리거나, 저장해둔 파일에서 로드하세요.


In [18]:
# 학습 시 사용했던 동일 버전 파일을 권장
df_original = pd.read_csv("./eda/data/merged_data.csv")
df_original.head()

,기준_년분기_코드,자치구_코드_명,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_률,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,...,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,20191,강남구,PC방,122.0,132.0,9.8,13.0,6.8,9.0,10.0,...,178522.0,164466.0,100728.0,44021.0,1679.0,99408.0,108205.0,74466.0,54523.0,22838.0
1,20191,강남구,가구,435.0,436.0,2.8,12.0,0.7,3.0,1.0,...,178522.0,164466.0,100728.0,44021.0,1679.0,99408.0,108205.0,74466.0,54523.0,22838.0
2,20191,강남구,가방,279.0,280.0,4.3,12.0,3.6,10.0,1.0,...,178522.0,164466.0,100728.0,44021.0,1679.0,99408.0,108205.0,74466.0,54523.0,22838.0
3,20191,강남구,가전제품,163.0,163.0,1.8,3.0,1.2,2.0,0.0,...,178522.0,164466.0,100728.0,44021.0,1679.0,99408.0,108205.0,74466.0,54523.0,22838.0
4,20191,강남구,가전제품수리,92.0,92.0,3.3,3.0,1.1,1.0,0.0,...,178522.0,164466.0,100728.0,44021.0,1679.0,99408.0,108205.0,74466.0,54523.0,22838.0


In [19]:
district_encoder = LabelEncoder()
industry_encoder = LabelEncoder()
sanggwon_encoder = LabelEncoder()

# ← 열 이름은 실제 csv의 컬럼명과 정확히 일치해야 합니다.
district_encoder.fit(df_original['자치구_코드_명'])
industry_encoder.fit(df_original['서비스_업종_코드_명'])
sanggwon_encoder.fit(df_original['상권_변화_지표'])

# classes_ 속성명(밑줄 중요)
print("District classes (n={}): {}".format(len(district_encoder.classes_), district_encoder.classes_[:10]))
print("Industry classes (n={}): {}".format(len(industry_encoder.classes_), industry_encoder.classes_[:10]))
print("Sanggwon classes (n={}): {}".format(len(sanggwon_encoder.classes_), sanggwon_encoder.classes_[:10]))

District classes (n=25): ['강남구' '강동구' '강북구' '강서구' '관악구' '광진구' '구로구' '금천구' '노원구' '도봉구']
Industry classes (n=63): ['PC방' '가구' '가방' '가전제품' '가전제품수리' '고시원' '골프연습장' '네일숍' '노래방' '당구장']
Sanggwon classes (n=4): ['HH' 'HL' 'LH' 'LL']


In [20]:
# 안전 가드: 필수 객체가 있는지 다시 한번 점검
for name in ["final_model", "X_train", "y_train", "X_test", "y_test"]:
    assert name in globals(), f"'{name}'가 커널에 없습니다. 먼저 학습하거나 로드하세요."

package = {
    "model": final_model,
    "district_encoder": district_encoder,
    "industry_encoder": industry_encoder,
    "sanggwon_encoder": sanggwon_encoder,
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
    "feature_names": X_train.columns.tolist()
}

joblib.dump(package, "model_and_data.pkl")
print("✅ 모델, 인코더, 데이터 저장 완료!")
print(f"Feature 개수: {len(package['feature_names'])}")


AssertionError: 'final_model'가 커널에 없습니다. 먼저 학습하거나 로드하세요.

In [ ]:
loaded = joblib.load("model_and_data.pkl")
print("🔎 Keys:", list(loaded.keys()))
print("features ({}):".format(len(loaded["feature_names"])), loaded["feature_names"][:8])
type(loaded["model"])
